In [1]:
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from matplotlib import gridspec

import sys
sys.path.insert(0, '/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/')

from trainlib.ConfigFileHandler import ConfigFileHandler

In [2]:
def load_file(path, keys):
    confhandler = ConfigFileHandler()
    confhandler.load_configuration(path)
    
    retval = {}
    
    for section_name in confhandler.get_sections():
        cur_section = confhandler.get_section(section_name)
        
        for key in keys:
            if not key in retval:
                retval[key] = []
                
            retval[key].append(float(cur_section[key]))
            
    return retval

In [20]:
def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    return mu, sigma

def plot_gp(bo, x):
    
    fig = plt.figure()
    axis = plt.subplot(111)
    
    mu, sigma = posterior(bo, x)
    axis.plot(bo.X.flatten(), bo.Y, 's', markersize=3, label=u'observations', color='black')
    axis.plot(x, mu, '--', color='black', label='gaussian process mean')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='lightblue', ec='None', label='95% confidence interval')
    
    axis.set_xlim((min(x), max(x)))
    axis.set_ylim((None, None))
    axis.set_ylabel(r'$u$')
    axis.set_xlabel(r'$p(\mathrm{ggH})/p(\mathrm{VBF})$')
    
    axis.legend(loc = 3)
    
    return fig

In [4]:
init_dict = load_file("/data_CMS/cms/wind/180528_optimized_hyperparameters_selvars_leading_jets_metfix_merged/optimized/priors_tree/evaluations_ggH.txt", ["target", "ggh_prior"])

attempting to load configuration file from /data_CMS/cms/wind/180528_optimized_hyperparameters_selvars_leading_jets_metfix_merged/optimized/priors_tree/evaluations_ggH.txt


In [5]:
gp_params = {'kernel': 1.0 * Matern(length_scale = 0.01, length_scale_bounds = (1e-5, 1e5), nu = 1.5),
             'alpha':1.5e-6}

In [6]:
target = lambda x: x

In [7]:
x = np.linspace(1.0, 3.0, 1000).reshape(-1,1)

In [8]:
bo = BayesianOptimization(target, {'ggh_prior': (1.0, 3.0)})

In [9]:
bo.initialize(init_dict)

In [14]:
bo.maximize(init_points=0, n_iter=0, acq='poi', kappa=5, xi = 0.1, **gp_params)

Bayesian Optimization
-------------------------------------------
 Step |   Time |      Value |   ggh_prior | 


In [21]:
fig = plot_gp(bo, x)

In [22]:
plt.savefig("/data_CMS/cms/wind/Plots/ggH_prior_optimization_tree_180531.pdf")